In [1]:
import numpy as np

from os.path import dirname, join

from bokeh.io import curdoc
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, Slider, TextInput, Toggle, Button, RadioButtonGroup, CustomJS
from bokeh.models import PreText, CheckboxGroup, Paragraph
from bokeh.plotting import figure
from bokeh.models.widgets import Tabs, Panel

from bokeh.models import Range1d

from bokeh.driving import count

import time
import serial

useSerial = True

if useSerial == True:
    # Estalish a connection with the CD48 board
    s = serial.Serial('/dev/cu.usbmodem14101',baudrate=250000, timeout = 1) # open serial port; port name will 
                                                            # have to be changed on different computers


# Define a class of objects called Counter. Each counter (0-7) will get a widget checkbox 
# for Port A through Port D   
class Counter(object):
    def __init__(self, portA, portB, portC, portD):
        self.portA = portA
        self.portB = portB
        self.portC = portC
        self.portD = portD
    

# Set a variable for common height of widgets
widgetHeight = 20   


# Create Label Widgets for each Counter Row. Ex) the 0th row will say "Counter 0:"
counter_labels = []

for i in range(8):
    counter_labels.append(PreText(text='Counter: ' + str(i), width = 80, height = widgetHeight))
    
# define width of port labels
portLabelWidth = 30

# Create Label widgets for each Port Column
portA = PreText(text='A: ', width = portLabelWidth, height = widgetHeight)
portB = PreText(text='B: ', width = portLabelWidth, height = widgetHeight)
portC = PreText(text='C: ', width = portLabelWidth, height = widgetHeight)
portD = PreText(text='D: ', width = portLabelWidth, height = widgetHeight)


# create some blank widgets just to "take up space" so the layout looks nicer
blank = PreText(text='', width = 80, height = widgetHeight)
blank2 = PreText(text='', width = 10, height = widgetHeight)
blank3 = PreText(text='', width = 20, height = widgetHeight)

# Create Set Button to send counter information to CD48 board
set_button = Button(
    label='Set Counters',
    disabled=False,
    button_type='success' # 'success', 'info', 'warning', 'danger' or ''
)



# define width of checkbox widgets
CheckboxWidth = 30

# Create the list counters. Each element of counters is an object of the class Counter.
# This means each element of counters will have 4 widget checkboxes, each referred to as portA - portD
# Later, you can call the "activeness"of say counter 4, port D checkbox in this way: 
# counters[4].portD.active   
# If counters[4].portD.active = [0], then this box is checked. 
# If counters[4].portD.active = [], then this box is not checked.

counters = []

# loop through 8 times to create 4 widget checkboxes (portA, portB, portC, portD) 
# for each of counters 0 through 7
for i in range(8):
    counters.append(Counter(CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight),
                           CheckboxGroup(active=[], 
                                            labels=[''], 
                                            width = CheckboxWidth, 
                                            height = widgetHeight)))
    

# create a list to store board settings in 
boardSettings = []

# the first element of boardSettings will just be the title "Board Settings:"
boardSettings.append(PreText(text="""Board Settings:""", width = 30, height = widgetHeight))

# for the remaining 12 items, just append blank ' ' which can be changed later
for i in np.arange(12):
    boardSettings.append(PreText(text=""" """, width = 30, height = widgetHeight))


# Define a function that can be called later. This function sends the 'P' command to the board, 
# then determines whether a given counter is currently "watching" a given port. 
# Then set the checkbox values accordingly (blah.active = [0] --> True, blah.active = [] --> False), 
# checkboxes are booleans
def set_checkboxes():
    
    # get the board settings
    s.write('P'.encode())
    time.sleep(0.5)
    settings = s.readlines()
    
    # add the board settings to the list boardSettings
    for i in np.arange(len(settings)):
        boardSettings[i+1].text = str(settings[i])
    
    # loop through 8 times to glean port settings for each counter
    for i in range(8):
        # for debugging purposes
        #print('counter = ', i)
        #print(settings[i][2:3])
        #print(settings[i][3:4])
        #print(settings[i][4:5])
        #print(settings[i][5:6])
        
        # the board writes back in bytes, not strings, hence the b'0' or b'1'.
        if settings[i][2:3] == b'0':
            counters[i].portA.active = []
        else:
            counters[i].portA.active = [0]
        
        if settings[i][3:4] == b'0':
            counters[i].portB.active = []
        else:
            counters[i].portB.active = [0]

        if settings[i][4:5] == b'0':
            counters[i].portC.active = []
        else:
            counters[i].portC.active = [0]

        if settings[i][5:6] == b'0':
            counters[i].portD.active = []
        else:
            counters[i].portD.active = [0]

# run the set_checkboxes function to give the checkboxes proper 
# intial values that reflect the current board settings
set_checkboxes()



# Create info labels that can warn the user if an error is made (you cannot clear a counter)
infoLabels = []

# append blank text ' ' that can be changed later
for i in range(8):
    infoLabels.append(PreText(text='', width = 400, height = widgetHeight))

    
# Create one horizontal box for each counter, which includes the port checkboxes
counterBoxes = []

for i in range(8):
    counterBoxes.append(row([counter_labels[i], 
                               counters[i].portA, 
                               counters[i].portB, 
                               counters[i].portC, 
                               counters[i].portD, 
                               blank2,
                               infoLabels[i]]))
    


# create a horizontal box with the port column labels
portsBox = row([blank, portA, portB, portC, portD])

# display our row with button and info_sent label
buttonRow = row([blank3, set_button])


# create a column of text widgets to display the board settings
boardSettingsColumn = column([boardSettings[0],
                              boardSettings[1],
                              boardSettings[2],
                              boardSettings[3],
                              boardSettings[4],
                              boardSettings[5],
                              boardSettings[6],
                              boardSettings[7],
                              boardSettings[8],
                              boardSettings[9],
                              boardSettings[10],
                              boardSettings[11],
                              boardSettings[12]])

# create the layout of our interface including all the rows and columns above
counterInterface = row([column([portsBox, 
                          counterBoxes[0], 
                          counterBoxes[1], 
                          counterBoxes[2], 
                          counterBoxes[3],
                          counterBoxes[4],
                          counterBoxes[5],
                          counterBoxes[6],
                          counterBoxes[7],
                          buttonRow]), boardSettingsColumn])


# define a function that will be run every time the Set Counters button is clicked
def send_info():
    
    # loop through 8 times, one for each counter
    for i in range(8):
        # set the values A-D to 0
        A = 0
        B = 0
        C = 0
        D = 0
        # clear the warning label
        infoLabels[i].text = ''
        # check the checkbox values of portA - portD, and change respective values as needed
        if counters[i].portA.active == [0]:
            A = 1
        if counters[i].portB.active == [0]:
            B = 1
        if counters[i].portC.active == [0]:
            C = 1
        if counters[i].portD.active == [0]:
            D = 1
            
        # if the sum of A - D is at least 1 (aka at least one port box is checked/True)
        # then write the counter setting command to the board
        if A + B + C + D > 0:
            countingCommand = 'S'+ str(i) + str(int(A)) + str(int(B)) + str(int(C)) + str(int(D)) + '\n'
            s.write(countingCommand.encode())
        # else (if the sum of A - D is 0), warn the user that this is not permitted. Do not send command to board
        else:
            infoLabels[i].text = 'Counter ' + str(i) + ' must watch at least 1 port.'
    
    # run the set_checkboxes function to ensure that the values of the checkboxes 
    # reflect the current board settings
    set_checkboxes()


    
# call the send_info function when the button is clicked.
set_button.on_click(send_info)



# This portion of the code is for running the experiment


# open serial connection to microwave source
mw = serial.Serial('/dev/cu.usbserial-DA009KJC', baudrate=9600, timeout = 1)




# define lf (low frequency)
lf = 2850000


    
    
# Initialize the data lists, time is the same for all three plots
# use a Column Data Source, intially all lists are empty

source = ColumnDataSource(dict(timeList=[],
                               frequencyList = [],
                               singleCountsA=[]
                              ))






# Set up user widgets

# Set up slider for user to choose counting time (100ms to 1000ms)
countingTimeSlider = Slider(start=0.1, end=20, value=1, step=0.1, title="Counting Time (s)")
countingTime = countingTimeSlider.value # initialize the variable to store the counting interval


# Set up START/STOP button, to start and stop counting
startStopButtonGroup = RadioButtonGroup(labels=["START", "STOP"], active=1)

# Set up a run experiment button. when clicked, experiment will run from start to finish
runExperimentButton = Button(label = 'Run Experiment', button_type = 'primary')

# Set up slider for user to set frequency range
frequencyRangeLowerSlider = Slider(start=2.80, end=2.95, value=2.84, step=0.01, title="Frequency Range Start (GHz)")
frequencyRangeLower = frequencyRangeLowerSlider.value # initialize the variable to store the frequency range

# Set up slider for user to set frequency range
frequencyRangeUpperSlider = Slider(start=2.85, end=3.0, value=2.90, step=0.01, title="Frequency Range End (GHz)")
frequencyRangeUpper = frequencyRangeUpperSlider.value # initialize the variable to store the frequency range

# Set up slider for user to set frequency step
frequencyStepSlider = Slider(start= 0.1, end=2, value=1, step=0.1, title="Frequency Step (MHz)")

# Set up the reset button, to clear the graph of all data
resetGraphButton = Button(label="Reset")

# Set up the save datda button, to export the data to a csv file
saveDataButton = Button(label="Save Data")

frequencyLabel = PreText(text='lf = ', width = portLabelWidth, height = widgetHeight)

AHover = [
    ("Frequency: ", "@frequencyList"),
    ("Counts: ", "@singleCountsA")
]




# Create our 4 plots



singleCountsAPlot = figure(plot_height=500, plot_width=1000, title="Single Counts on Port A",
              tools="crosshair,pan,reset,save,wheel_zoom", x_range = (frequencyRangeLower, frequencyRangeUpper), tooltips = AHover)


# Set y-axis labels for graphs
singleCountsAPlot.yaxis.axis_label = "Counts per " + str(countingTime) + " seconds"



# How many points should be displayed on screen at a time. Can turn this into a user input widget
numberPointsDisplayed = 20



# Make each point of each plot a circle

singleCountsAPlot.circle(x = 'frequencyList', 
                         y = 'singleCountsA', 
                         source = source, 
                         line_width=3, 
                         line_alpha=0.6
                         )


# Connect the points of each plot with a line

singleCountsAPlot.line(x = 'frequencyList', 
                       y = 'singleCountsA', 
                       source = source, 
                       line_width=2, 
                       line_alpha=0.6
                       )



# set global variables to 0

# keep a running count of time elapsed, start at 0. This will help inform x-axis values.
runningTime = 0

numberDataPoints = 0

outputfrequency = 20

totalNumberDataPoints = 0

countingTime = 0
frequencyRange = 0
frequencyStep = 0

# Define a variable callback_id, which is attached to the periodic_callback function
callback_id = None

# pause for 12 seconds
#time.sleep(12)


# Define a fuction to call when the current document (curdoc) has periodic callback on
# This function appends new data to our plots through the stream data command
def update_data():
    
    global numberDataPoints
    global totalNumberDataPoints
    global frequencyRangeUpper
    global frequencyRangeLower
    global lf
    
    # add data if the current frequency is less than the maximum frequency set by the frequencyRange slider
    if lf <= (frequencyRangeUpper * 10**6):
            
        # use the global variable
        global countingTime
        global frequencyStep
        global runningTime
        global outputfrequency
        
        countingTime = countingTimeSlider.value

        # add the counting time to our running time
        runningTime = runningTime + countingTime
        
        # Request data from the CD48 counter
        s.write("c\n".encode())
        serialData = s.readline()
        counterData = [int(x) for x in serialData.decode('ascii').rstrip().split(' ')]

        # append one new data point to each list
        new_data = dict(
            timeList=[runningTime],
            frequencyList = [lf * 10**(-6)],
            singleCountsA=[counterData[0]]
            )

        # Stream the new data to the plots.
        source.stream(new_data)
        
        frequencyLabel.text='lf = ' + str(lf)
        
        # increase lf by the frequencyStep
        lf = lf + frequencyStep * 10**(3)
        
        # send a command to the mw controller setting the low frequency to lf
        lfstr = str(lf)
        lfstr = 'LF ' + lfstr + ';'
        mw.write(lfstr.encode())
    
    # else don't add new data
    else:
        
        # turn off the microwave source
        mw.write('ON 0;'.encode())
        
        # remove callback
        global callback_id
        curdoc().remove_periodic_callback(callback_id)

        
        
def run_experiment():
    
    global callback_id
    global numberDataPoints
    global outputfrequency
    global totalNumberDataPoints
    global runningTime
    
    global countingTime
    global frequencyStep
    
    global lf
    
    global frequencyRangeLower
    global frequencyRangeUpper
    
    
    # retrieve slider values
    countingTime = countingTimeSlider.value
    frequencyStep = frequencyStepSlider.value
    
    
    
    frequencyRangeLower = frequencyRangeLowerSlider.value
    frequencyRangeUpper = frequencyRangeUpperSlider.value
    
    singleCountsAPlot.x_range.update(start = frequencyRangeLower, end = frequencyRangeUpper)
    
    
    # set the starting frequency
    lf = frequencyRangeLower * 10**6
    
    
    runningTime = 0
    
    
    # make a count to clear counters
    s.write("c\n".encode())
    serialData = s.readline()
    
    # clear overflow
    s.write("E\n".encode())
    serialData = s.readline()
    
    # set low frequency (in kHz)
    lfstr = str(lf)
    lfstr = 'LF ' + lfstr + ';'
    mw.write(lfstr.encode())

    # set high frequency (in kHz)
    mw.write('HF 2950000;'.encode())

    # set attenuation (in dB)
    mw.write('AT 0;'.encode())

    # set up and down sweep rates (in MHz/s)
    mw.write('UR 10;'.encode())
    mw.write('DR 10;'.encode())
    
    # turn on the microwave source
    mw.write('ON 1;'.encode())
    
    #curdoc().remove_periodic_callback(callback_id)
    callback_id = curdoc().add_periodic_callback(update_data, countingTime*1000)
    

# Define a function to be called when the reset button is pressed.   
def reset_plot():
    
    # set the running time back to 0
    global runningTime
    global numberDataPoints
    global outputfrequency
    global totalNumberDataPoints
    
    
    
    runningTime = 0
    
    numberDataPoints = 0

    outputfrequency = 0

    totalNumberDataPoints = 0
    
    # empty all data lists in source
    source.data = {k: [] for k in source.data}     
    
    global lf
    global frequencyRangeUpper
    
    lf = (2 * frequencyRangeUpper) * 10**6
    
    # turn off the microwave source
    mw.write('ON 0;'.encode())
    
    
    
# Change y axis labels based on counting time value
def change_yaxis_labels(attr, old, new):
    
    countingTime = countingTimeSlider.value
    
    # Set y-axis labels for graphs
    singleCountsAPlot.yaxis.axis_label = "Counts per " + str(float(round(countingTime,1))) + " seconds"
    
    


# When the reset button is clicked, call the reset_plot function
resetGraphButton.on_click(reset_plot)

# run experiment when button is clicked
runExperimentButton.on_click(run_experiment)

# change y axis labels when counting time value changes
countingTimeSlider.on_change('value', change_yaxis_labels)


# When the save data button is clicked, export the data into a csv file
saveDataButton.js_on_click(CustomJS(args=dict(source=source),
                            code=open(join(dirname(__file__), "download.js")).read()))

# Set up layouts and add to document

# create a column of user input widgets
userInputWidgets = column(runExperimentButton,
                          frequencyRangeLowerSlider,
                          frequencyRangeUpperSlider,
                          frequencyStepSlider,
                          countingTimeSlider, 
                          resetGraphButton,
                          saveDataButton,
                          frequencyLabel
                         )


# create a layout with the set counters interface from the first half of this code
setCountersLayout = layout([[counterInterface]], sizing_mode='scale_both')

# create a column with our three plots
plotsColumn = column([singleCountsAPlot
                     ])

# create a layout for the graphs interface, with the input widgets and the three plots
displayCountsLayout = layout([[row([userInputWidgets, plotsColumn])]] , sizing_mode='scale_both')

# create a tab for the set counters interface
setCountersTab = Panel(child = setCountersLayout, 
                       title="Set Counters")

# create a tab for the graphs interface
displayCountsTab = Panel(child = displayCountsLayout, 
                         title="Graphs")

# combine all tabs into one layout
currentDocumentTabs = Tabs(tabs=[ displayCountsTab, setCountersTab ])


# Add tabs to current document
curdoc().add_root(currentDocumentTabs)

curdoc().title = "ODMRInterface"

IndexError: list index out of range